In [35]:
import numpy as np
import pandas as pd

In [36]:
pd.set_option('display.max_columns', None)
PATH_DATA_RAW= '../Data/Raw/'
PATH_DATA_CLEAN = '../Data/Clean/'

## Load Data

In [37]:
data=pd.read_csv(PATH_DATA_RAW+'train.csv')
data.sample(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,bookingID
41179,"The Westin Peachtree Plaza, Atlanta Atlanta, GA",0,71,2018,August,33,12,1,2,2,0.0,0,BB,ESP,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,189.0,0,0,Check-Out,2018-08-15,41180
32485,"San Diego Sorrento Valley Courtyard San Diego, CA",0,55,2018,April,17,17,2,3,2,0.0,0,BB,IRL,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,177.0,NaN,0,Transient,43.4,0,0,Check-Out,2018-04-22,32486
78073,Newark Liberty International Airport Courtyard...,0,0,2018,August,35,26,0,1,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,E,E,0,No Deposit,NaN,NaN,0,Transient,169.0,0,0,Check-Out,2018-08-27,78074
374,"Orlando Airport Courtyard Orlando, FL",1,125,2018,December,53,27,0,3,3,0.0,0,BB,ITA,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,138.3,0,0,Canceled,2018-08-25,375
72631,Sheraton Grand Rio Hotel & Resort Rio de Janei...,0,37,2018,February,7,10,1,4,2,0.0,0,BB,GBR,Offline TA/TO,TA/TO,0,0,0,D,E,1,No Deposit,171.0,NaN,0,Transient,37.0,0,0,Check-Out,2018-02-15,72632


## Checking Columns and Missing Values

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83293 entries, 0 to 83292
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           83293 non-null  object 
 1   is_canceled                     83293 non-null  int64  
 2   lead_time                       83293 non-null  int64  
 3   arrival_date_year               83293 non-null  int64  
 4   arrival_date_month              83293 non-null  object 
 5   arrival_date_week_number        83293 non-null  int64  
 6   arrival_date_day_of_month       83293 non-null  int64  
 7   stays_in_weekend_nights         83293 non-null  int64  
 8   stays_in_week_nights            83293 non-null  int64  
 9   adults                          83293 non-null  int64  
 10  children                        83290 non-null  float64
 11  babies                          83293 non-null  int64  
 12  meal                            

Dari tipe data:\
Kolom children harusnya adalah int64 bukan float64 karena jumlah anak itu dihitung bukan diukur\
Kolom company dan agent adalah kolom ID, idealnya tipe datanya antara objek atau int\
Kolom reservation_status_date harusnya adalah datetime

Secara umum:\
Data tersebut memiliki 83293 baris dan 32 kolom\
Data tersebut memiliki missing value pada beberapa kolom\
Target untuk diprediksi adalah is_canceled\
Kolom selain is_canceled adalah fitur yang digunakan untuk memprediksi is_canceled

Selanjutnya ada akan dipisahkan antara kolom numerikal dan kategorikal

In [39]:
numerical = []
categorical = []
for i in data.columns:
    if (data[i].dtypes == 'float64') or (data[i].dtypes == 'int64'):
        numerical.append(i)
    elif (data[i].dtypes != 'float64') and (data[i].dtypes != 'int64'):
        categorical.append(i)
print(numerical)
print(categorical)

['is_canceled', 'lead_time', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'agent', 'company', 'days_in_waiting_list', 'adr', 'required_car_parking_spaces', 'total_of_special_requests', 'bookingID']
['hotel', 'arrival_date_month', 'meal', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type', 'reservation_status', 'reservation_status_date']


Statistical summary dengan

In [40]:
data[numerical].describe()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,bookingID
count,83293.000000,83293.000000,83293.000000,83293.000000,83293.000000,83293.000000,83293.000000,83293.000000,83290.000000,83293.000000,83293.000000,83293.000000,83293.000000,83293.000000,71889.000000,4734.000000,83293.000000,83293.000000,83293.000000,83293.000000,83293.000000
mean,0.369503,103.914230,2018.156808,27.164023,15.786116,0.930390,2.503812,1.854466,0.103614,0.008032,0.030243,0.086994,0.141765,0.223068,86.867282,190.003802,2.352238,101.724373,0.061818,0.571945,41647.000000
std,0.482673,106.874124,0.706656,13.623752,8.775931,1.004196,1.924511,0.606139,0.398821,0.100641,0.171255,0.862988,1.554390,0.656237,110.875605,131.969152,17.820652,48.082552,0.245027,0.793742,24044.762323
min,0.000000,0.000000,2017.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,18.000000,2018.000000,16.000000,8.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,62.000000,0.000000,69.530000,0.000000,0.000000,20824.000000
50%,0.000000,69.000000,2018.000000,27.000000,16.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,183.000000,0.000000,94.500000,0.000000,0.000000,41647.000000
75%,1.000000,160.000000,2019.000000,38.000000,23.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,229.000000,270.000000,0.000000,126.000000,0.000000,1.000000,62470.000000
max,1.000000,737.000000,2019.000000,53.000000,31.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,70.000000,21.000000,535.000000,543.000000,391.000000,508.000000,8.000000,5.000000,83293.000000


In [41]:
data[categorical].describe()

,hotel,arrival_date_month,meal,country,market_segment,distribution_channel,reserved_room_type,assigned_room_type,deposit_type,customer_type,reservation_status,reservation_status_date
count,83293,83293,83293,82947,83293,83293,83293,83293,83293,83293,83293,83293
unique,64,12,5,165,8,5,10,11,3,4,3,914
top,Renaissance New York Times Square Hotel New Yo...,August,BB,PRT,Online TA,TA/TO,A,A,No Deposit,Transient,Check-Out,2017-10-21
freq,1373,9643,64424,33952,39445,68368,60062,51702,72979,62535,52516,1024


Melihat secara menyeluruh nilai-nilai fitur kategorikal dan fitur numerikal diskrit

In [42]:
for col in categorical:
    print(f'''Value count kolom {col}:''')
    print('-' * 50)
    print(data[col].value_counts())
    print()

Value count kolom hotel:
--------------------------------------------------
hotel
Renaissance New York Times Square Hotel New York, NY                           1373
Sacramento Airport Natomas Courtyard Sacramento, CA                            1372
Las Vegas Marriott Las Vegas, NV                                               1371
Crystal Cove, Barbados Barbados                                                1360
Protea Hotel by Marriott Midrand Midrand, South Africa                         1352
                                                                               ... 
W New York – Union Square New York, NY                                         1250
The House, Barbados Barbados                                                   1249
Sheraton Grand Rio Hotel & Resort Rio de Janeiro, Brazil                       1249
African Pride Melrose Arch, Autograph Collection Johannesburg, South Africa    1246
Residence Inn Rio de Janeiro Barra da Tijuca Barra da Tijuca, Brazil          

In [43]:
diskrit = ['is_canceled', 'adults', 'children', 'babies', 'arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month',
'stays_in_weekend_nights', 'stays_in_week_nights', 'is_repeated_guest', 'previous_cancellations', 'previous_bookings_not_canceled',
'booking_changes', 'required_car_parking_spaces', 'total_of_special_requests']

for col in diskrit:
    print(f'''Value count kolom {col}:''')
    print('-' * 50)
    print(data[col].value_counts())
    print()

Value count kolom is_canceled:
--------------------------------------------------
is_canceled
0    52516
1    30777
Name: count, dtype: int64

Value count kolom adults:
--------------------------------------------------
adults
2     62373
1     16263
3      4316
0       287
4        41
26        5
50        1
10        1
27        1
40        1
55        1
5         1
6         1
20        1
Name: count, dtype: int64

Value count kolom children:
--------------------------------------------------
children
0.0     77329
1.0      3351
2.0      2558
3.0        51
10.0        1
Name: count, dtype: int64

Value count kolom babies:
--------------------------------------------------
babies
0     82650
1       632
2         9
10        1
9         1
Name: count, dtype: int64

Value count kolom arrival_date_year:
--------------------------------------------------
arrival_date_year
2018    39652
2019    28351
2017    15290
Name: count, dtype: int64

Value count kolom arrival_date_week_number:
---

## Data Cleansing

Missing Value Handling

In [44]:
data_clean = data.copy()

In [45]:
data_missing_value = data_clean[data_clean.columns[data_clean.isna().any()]].isnull().sum().reset_index()
data_missing_value.columns = ['fitur','missing_count']
data_missing_value['percentage'] = round((data_missing_value['missing_count']/len(data_clean))*100,3)
data_missing_value = data_missing_value.sort_values('percentage', ascending=False).reset_index(drop=True)
data_missing_value

,fitur,missing_count,percentage
0,company,78559,94.316
1,agent,11404,13.691
2,country,346,0.415
3,children,3,0.004


Ada 4 kolom yang memiliki missing value, yaitu:
1. Company
2. Agent
3. Country
4. Children

- Kolom company karena memiliki missing value yang terlalu banyak, maka kolomnya akan di drop
- Kolom agent akan diisi dengan 0 dengan asumsi bahwa orang yang reservasi tidak melewati agent
- Kolom country dan children karena memiliki persentase missing value yang sangat sedikit maka baris2 yang mengandung missing value tersebut akan dihapus

In [46]:
data_clean.drop(columns=['company'], inplace=True)

In [47]:
data_clean.dropna(subset=['children'], inplace=True)
data_clean.dropna(subset=['country'], inplace=True)
data_clean['agent'] = data_clean['agent'].apply(lambda x: 0 if pd.isnull(x) else x)

In [48]:
data_clean.sample(2)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,bookingID
65001,Residence Inn Las Vegas Convention Center Las ...,1,196,2018,August,32,6,1,1,3,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,0,Transient,128.7,0,0,Canceled,2018-04-16,65002
20035,Sheraton Mexico City Maria Isabel Hotel Mexico...,1,183,2019,June,23,8,0,2,1,0.0,0,BB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,Non Refund,86.0,0,Transient,120.0,0,0,Canceled,2018-12-07,20036


In [49]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82944 entries, 0 to 83292
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hotel                           82944 non-null  object 
 1   is_canceled                     82944 non-null  int64  
 2   lead_time                       82944 non-null  int64  
 3   arrival_date_year               82944 non-null  int64  
 4   arrival_date_month              82944 non-null  object 
 5   arrival_date_week_number        82944 non-null  int64  
 6   arrival_date_day_of_month       82944 non-null  int64  
 7   stays_in_weekend_nights         82944 non-null  int64  
 8   stays_in_week_nights            82944 non-null  int64  
 9   adults                          82944 non-null  int64  
 10  children                        82944 non-null  float64
 11  babies                          82944 non-null  int64  
 12  meal                            82944

Duplicate value handling

In [50]:
data_clean.duplicated().sum()

0

Tidak ada data duplikat

Adjust Data Type

In [51]:
# Changing children to integer type: 
data_clean['children'] = data_clean['children'].astype(int)

# Changing reservation status date to datetime type: 
data_clean['reservation_status_date'] = pd.to_datetime(data['reservation_status_date'])

# Changing agent to integer type: 
data_clean['agent'] = data_clean['agent'].astype(int)

Handle Incorrect Value

In [52]:
data_clean['adults'].describe()

count    82944.000000
mean         1.856337
std          0.605626
min          0.000000
25%          2.000000
50%          2.000000
75%          2.000000
max         55.000000
Name: adults, dtype: float64

Nilai 0 pada adults ini adalah hal yang tidak masuk akal, karena memesan hotel minimal harus dipesan untuk satu orang dewasa\
Baris dengan adulst sama dengan nol akan didrop

In [53]:
data_clean.drop(data_clean[data_clean['adults'] < 1].index, inplace = True)

In [54]:
data_clean['adults'].describe()

count    82665.000000
mean         1.862602
std          0.596952
min          1.000000
25%          2.000000
50%          2.000000
75%          2.000000
max         55.000000
Name: adults, dtype: float64

In [55]:
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82665 entries, 0 to 83292
Data columns (total 32 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   hotel                           82665 non-null  object        
 1   is_canceled                     82665 non-null  int64         
 2   lead_time                       82665 non-null  int64         
 3   arrival_date_year               82665 non-null  int64         
 4   arrival_date_month              82665 non-null  object        
 5   arrival_date_week_number        82665 non-null  int64         
 6   arrival_date_day_of_month       82665 non-null  int64         
 7   stays_in_weekend_nights         82665 non-null  int64         
 8   stays_in_week_nights            82665 non-null  int64         
 9   adults                          82665 non-null  int64         
 10  children                        82665 non-null  int32         
 11  babies 

In [56]:
data_clean.to_csv(PATH_DATA_CLEAN+'train_clean.csv',index = False)